# 1. 数据结构与算法

## 1.1 将序列分解为单独的变量

In [1]:
data = ['ACME', 50, 91.1, (2020, 12, 21)]
# 任何序列（或可迭代的对象）都可以通过一个简单的赋值操作来分解为单独的变量。
# 要求变量的总数和结构要与序列相吻合
name, shares, price, date = data 
name

'ACME'

In [2]:
date

(2020, 12, 21)

In [3]:
name, shares, price, (year, mon, day) = data
year

2020

In [4]:
s = 'hello' # 只要对象是可迭代的就可以执行分解操作，包括字符串、文件、迭代器以及生成器
a, b, c, d, e = s
a

'h'

In [5]:
_, shares, price, _ = data # 选一个用不到的变量名，作为要丢弃的值的名称
shares

50

## 1.2 从任意长度的可迭代对象中分解元素

In [6]:
record = ['Dave', '773-555-1111', '773-555-1112', 'dave@example.com']
name, *phonenumbers, email = record # 使用 Python 的“*表达式”
email

'dave@example.com'

In [7]:
phonenumbers

['773-555-1111', '773-555-1112']

In [8]:
# *表达式 的语法在迭代一个变长的元组序列时尤其有用
records = [('foo', 1, 2), ('bar', 'hello'), ('foo', 3 ,4)]

def do_foo(x, y):
    print('foo', x, y)
    
def do_bar(s):
    print('bar', s)

for tag, *args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

foo 1 2
bar hello
foo 3 4


In [9]:
# 和某些特定的字符串处理操作（例如拆分）相结合
line = 'nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false'
uname, *fields, homedir, sh = line.split(':')
fields

['*', '-2', '-2', 'Unprivileged User']

In [10]:
sh

'/usr/bin/false'

In [11]:
name, *_, (*_, day) = data # 丢弃多个值
day

21

In [12]:
items = [1, 2, 4, 6, 8] # 将列表分解为头部和尾部
head, *tail = items
tail

[2, 4, 6, 8]

## 1.3 保存最后N个元素

In [ ]:
# 使用collections.deque保存有限的历史记录
# 对一系列文本行做简单的文本匹配操作，当发现有匹配时就输出当前的匹配行以及最后检查过的N行文本
from collections import deque

def search(lines, pattern, history = 5):
    previous_lines = deque(maxlen = history) # 创建了一个固定长度的队列
    for line in lines:
        if pattern in lines:
            yield line, previous_lines 
            # yield 的作用就是把一个函数变成一个 generator，带有 yield 的函数不再是一个普通函数，Python 解释器会将其视为一个 generator
            # 每执行到一个 yield 语句就会中断，并返回一个迭代值，下次执行时从 yield 的下一个语句继续执行
        previous_lines.append(line)
        
if __name__ == '__main__':
    with open('somefile.txt') as f:
        for line, prevlines in search(f, 'python', 5):
            for pline in prevlines:
                print(pline, end = '')
                print('-'*20)

# python中yield的用法详解 https://blog.csdn.net/mieleizhi0522/article/details/82142856

In [14]:
from collections import deque
q = deque()
q.append(1)
q.append(2)
q

deque([1, 2])

In [15]:
q.appendleft(4)
q

deque([4, 1, 2])

In [16]:
q.pop()

2

In [17]:
q.popleft() # 从队列两端添加或者弹出元素的复杂度都是O(1)，从列表的头部插入或移除元素的复杂度是O(n)

4

In [18]:
q

deque([1])

## 找到最大或最小的N个元素

In [19]:
# heapq 模块中的nlargest()和nsmallest()函数,这两个函数可以接受一个参数key，从而允许在更加复杂的数据结构中使用
import heapq
nums = [1, 3, 2, 0, -1, 8, 5]
print(heapq.nlargest(3, nums))
print(heapq.nsmallest(2, nums))

[8, 5, 3]
[-1, 0]


In [21]:
import heapq

portfolio = [
   {'name': 'IBM', 'shares': 100, 'price': 91.1},
   {'name': 'AAPL', 'shares': 50, 'price': 543.22},
   {'name': 'FB', 'shares': 200, 'price': 21.09},
   {'name': 'HPQ', 'shares': 35, 'price': 31.75},
   {'name': 'YHOO', 'shares': 45, 'price': 16.35},
   {'name': 'ACME', 'shares': 75, 'price': 115.65}
]

cheap = heapq.nsmallest(3, portfolio, key=lambda s: s['price'])
expensive = heapq.nlargest(3, portfolio, key=lambda s: s['price'])

print(cheap)
print(expensive)

[{'name': 'YHOO', 'shares': 45, 'price': 16.35}, {'name': 'FB', 'shares': 200, 'price': 21.09}, {'name': 'HPQ', 'shares': 35, 'price': 31.75}]
[{'name': 'AAPL', 'shares': 50, 'price': 543.22}, {'name': 'ACME', 'shares': 75, 'price': 115.65}, {'name': 'IBM', 'shares': 100, 'price': 91.1}]


## 实现优先级队列

In [23]:
import heapq

class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._index = 0
        
    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))
        self._index += 1
    
    def pop(self):
        return heapq.heappop(self._queue)[-1]

class Item:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return 'Item({!r})'.format(self.name)


q = PriorityQueue()
q.push(Item('foo'), 1)
q.push(Item('bar'), 5)
q.push(Item('spam'), 4)
q.push(Item('grok'), 1)
q.pop()

Item('bar')